## <center> Basic Hello World </center>

In [ ]:
import pika

In [ ]:
connection = pika.BlockingConnection(pika.ConnectionParameters(host="localhost"))

In [ ]:
channel = connection.channel()

In [ ]:
channel.queue_declare(queue="hello")

In [ ]:
def callback(ch, method, properties, body):
    print(body,"received...")
    

In [ ]:
channel.basic_consume(queue="hello", on_message_callback=callback, auto_ack=True)

In [ ]:
print("waiting for messages...")
channel.start_consuming()

## <center> Worker </center>

In [ ]:
import pika
import time

In [ ]:
connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.queue_declare(queue='task_queue', durable=True)
print(' [*] Waiting for messages. To exit press CTRL+C')

In [ ]:
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body.decode())
    time.sleep(body.count(b'.'))
    print(" [x] Done")
    ch.basic_ack(delivery_tag=method.delivery_tag)

In [ ]:
channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='task_queue', on_message_callback=callback)

channel.start_consuming()

## <center> Pub-Sub </center>

In [ ]:
import pika

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='logs', exchange_type='fanout')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

channel.queue_bind(exchange='logs', queue=queue_name)

print(' [*] Waiting for logs. To exit press CTRL+C')

def callback(ch, method, properties, body):
    print(" [x] %r" % body)

channel.basic_consume(
    queue=queue_name, on_message_callback=callback, auto_ack=True)

channel.start_consuming()